### Map Paths to Resource titles and links
    1. Get csv and convert to df
    1. Parse the mapping 
       1. create a list of words in new columns
    2. replace path with title + links


### Get CSV and convert to df

In [1]:
import pandas
from pathlib import Path

In [2]:
out_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-alerts/input/data/DirectToDaVinciMap.csv')
in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-alerts/input/images/source/DirectNotificationsToDaVinciNotificationsMapping.xlsx')
df = pandas.read_excel(in_path, sheet_name='Element Mapping', keep_default_na=False)
df

,Data element name,Usage [Cardinality],Segment Name,Field Identifier,HL7 V2 Field Name,"V2 value set, other notes",FHIR element location,Usage [Cardinality].1,"FHIR Value set, other notes"
0,Message Code,R [1..1],MSH,9.1,Message Type.Message Code,constant 'ADT',not applicable,,
1,Message Event,R [1..1],MSH,9.2,Message Type.Trigger Event,Allowable values from https://terminology.hl7....,https://hl7.org/fhir/us/davinci-alerts/2020FEB...,R [1..1],https://hl7.org/fhir/us/davinci-alerts/2020FEB...
2,Transaction ID,R [1..1],MSH,10,Message Control ID,,https://hl7.org/fhir/us/davinci-alerts/2020FEB...,R [1..1],
3,Version ID,R [1..1],MSH,12,Version ID,Constant '2.5.1',not applicable,,
4,Event Date/Time,R [1..1],EVN,2.1,Recorded Date/Time - Time,Actual date and time of the event.\nThis guide...,For Admission (MSH-9.2 = A01 or A10) https://h...,RE [0..1],Mapping depends on the type of notification; i...
...,...,...,...,...,...,...,...,...,...
89,Patient Insurance Member ID,RE,IN1,36,IN1-36 - Policy Number,The V2-FHIR Mapping project didn't yet map this,https://hl7.org/fhir/us/davinci-alerts/Structu...,O [0..*],
90,Insured’s ID Number,RE,IN1,49,IN1-49 – Insured’s ID Number,,https://hl7.org/fhir/us/davinci-alerts/2020FEB...,R [0..1],
91,Mother's Name,RE [0..1],NK1,2,NK Name,when NK1-3 is valued MTH^Mother^HL70063,https://hl7.org/fhir/us/core/StructureDefiniti...,R [0..1],"US Core V3.1.0, which is what DaVinci Notifica..."
92,Mother's Address,RE [0..1],NK1,4,Address,when NK1-3 is valued MTH^Mother^HL70063,https://hl7.org/fhir/us/core/StructureDefiniti...,R [0..1],"US Core V3.1.0, which is what DaVinci Notifica..."


### Parse 'FHIR element location' as list and add to df

In [3]:
df['parsed'] = df['FHIR element location'].str.split(' ')
pandas.set_option('display.max_colwidth', None)
df['parsed']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 [not, applicable]
1                                                                                                                                                                                                                                                                                                                                                                                                                                   

### map the path to title with links

#### create a mapping table from path to titles

In [4]:
from json import loads
# Map Alerts Profiles titles to relative paths
alerts_sd_json = Path(r'/Users/ehaas/Documents/FHIR/davinci-alerts/temp/pages/_data/structuredefinitions.json')
alerts_sd_json = loads(alerts_sd_json.read_text())
# print(alerts_path_map)
my_map =  {v['path']:v['title'] for k,v in alerts_sd_json.items() if k != "maturities"}
my_map

{'StructureDefinition-admit-notification-messageheader.html': 'Da Vinci Admit Notification MessageHeader Profile',
 'StructureDefinition-adt-notification-condition.html': 'Da Vinci Admit/Discharge/Transfer Notification Condition Profile',
 'StructureDefinition-adt-notification-coverage.html': 'Da Vinci Admit/Discharge/Transfer Notification Coverage Profile',
 'StructureDefinition-adt-notification-encounter.html': 'Da Vinci Admit/Discharge/Transfer Notification Encounter Profile',
 'StructureDefinition-discharge-notification-messageheader.html': 'Da Vinci Discharge Notification MessageHeader Profile',
 'StructureDefinition-notifications-bundle.html': 'Da Vinci Notifications Bundle Profile',
 'StructureDefinition-notifications-messageheader.html': 'Da Vinci Notifications MessageHeader Profile',
 'StructureDefinition-transfer-notification-messageheader.html': 'Da Vinci Transfer Notification MessageHeader Profile'}

In [5]:
def title_mapper(uscore_package, uscore_base, my_map):
    for file in uscore_package.glob("*.json"):
        print(file)
        my_obj = loads(file.read_text())
        try:
            print(f'my_obj["resourceType"] = {my_obj["resourceType"]}')
            my_map[f'{uscore_base}{my_obj["resourceType"]}-{my_obj["id"]}.html']=my_obj["title"]   
        except KeyError:
            pass
    return(my_map)

# Map US Core 3.1.1 Profiles titles to absolute paths
uscore_package = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package')
uscore_base = 'https://hl7.org/fhir/us/core/STU3.1/'
# read all the json files and construct path from Type-id and get title if present
title_mapper(uscore_package, uscore_base, my_map)
# Map US Core 7.0.0 Profiles titles to absolute paths
uscore_package = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.us.core#7.0.0/package')
uscore_base = 'https://hl7.org/fhir/us/core/'
# read all the json files and construct path from Type-id and get title if present
title_mapper(uscore_package, uscore_base, my_map)
my_map 

/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/StructureDefinition-us-core-observation-lab.json
my_obj["resourceType"] = StructureDefinition
/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/ValueSet-detailed-ethnicity.json
my_obj["resourceType"] = ValueSet
/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/SearchParameter-us-core-immunization-status.json
my_obj["resourceType"] = SearchParameter
/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/SearchParameter-us-core-condition-category.json
my_obj["resourceType"] = SearchParameter
/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/SearchParameter-us-core-patient-identifier.json
my_obj["resourceType"] = SearchParameter
/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/SearchParameter-us-core-practitioner-identifier.json
my_obj["resourceType"] = SearchParameter
/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/CapabilityStatement-us-core-client.json
my_obj["resourceType"] =

{'StructureDefinition-admit-notification-messageheader.html': 'Da Vinci Admit Notification MessageHeader Profile',
 'StructureDefinition-adt-notification-condition.html': 'Da Vinci Admit/Discharge/Transfer Notification Condition Profile',
 'StructureDefinition-adt-notification-coverage.html': 'Da Vinci Admit/Discharge/Transfer Notification Coverage Profile',
 'StructureDefinition-adt-notification-encounter.html': 'Da Vinci Admit/Discharge/Transfer Notification Encounter Profile',
 'StructureDefinition-discharge-notification-messageheader.html': 'Da Vinci Discharge Notification MessageHeader Profile',
 'StructureDefinition-notifications-bundle.html': 'Da Vinci Notifications Bundle Profile',
 'StructureDefinition-notifications-messageheader.html': 'Da Vinci Notifications MessageHeader Profile',
 'StructureDefinition-transfer-notification-messageheader.html': 'Da Vinci Transfer Notification MessageHeader Profile',
 'https://hl7.org/fhir/us/core/STU3.1/StructureDefinition-us-core-observati

In [6]:
def get_rel_path(token):
    return token.split('/')[-1].replace('-definitions','').split('#')[0].replace('admit-discharge','adt')


def get_abs_path(token):
    return token.replace('-definitions','').split('#')[0]


def get_element_path(token):
    return token.split('#')[-1].split('.', 1)[-1]


for parsed in df['parsed']:
    # print(f'parsed = {parsed}')
    
    try:
        for i,token in enumerate(parsed):
            # print(f'i = {i}')
            if token.startswith('https://hl7.org/fhir/us/davinci-alerts/'):
                print(f'PATH! = {token}')
                parsed[i] = f'<a href="{get_rel_path(token)}">{my_map[get_rel_path(token)]}</a>.{get_element_path(token)}'
                print(f'as a link; {parsed[i]}')

            elif token.startswith('https://hl7.org/fhir/us/core/') or token.startswith('http://hl7.org/fhir/us/core/'):
                print(f'PATH! = {token}')
                parsed[i] = f'<a href="{get_abs_path(token)}">{my_map[get_abs_path(token)]}</a>.{get_element_path(token)}'
                print(f'as a link; {parsed[i]}')
    except TypeError as e:
       print(f'TypeError: {e}')
    except KeyError as e:
       print(f'KeyError: {e}')

PATH! = https://hl7.org/fhir/us/davinci-alerts/2020FEB/StructureDefinition-notifications-messageheader-definitions.html#MessageHeader.event[x]:eventCoding
as a link; <a href="StructureDefinition-notifications-messageheader.html">Da Vinci Notifications MessageHeader Profile</a>.event[x]:eventCoding
PATH! = https://hl7.org/fhir/us/davinci-alerts/2020FEB/StructureDefinition-notifications-bundle-definitions.html#Bundle.identifier.value
as a link; <a href="StructureDefinition-notifications-bundle.html">Da Vinci Notifications Bundle Profile</a>.identifier.value
PATH! = https://hl7.org/fhir/us/davinci-alerts/2020FEB/StructureDefinition-notifications-bundle-definitions.html#Bundle.identifier.system
as a link; <a href="StructureDefinition-notifications-bundle.html">Da Vinci Notifications Bundle Profile</a>.identifier.system
PATH! = https://hl7.org/fhir/us/davinci-alerts/2020FEB/StructureDefinition-admit-discharge-notification-encounter-definitions.html#Encounter.period.start
For
as a link; <a h

### Create a new column "joined" for updated mapping 

In [7]:
df['joined'] = df['parsed'].apply(lambda x: ' '.join(map(str, x)))

df.to_csv(out_path)